In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
from os import fdopen, remove, walk
import glob
from tempfile import mkstemp
import shutil
from shutil import move, copymode

In [3]:
from datetime import date

In [4]:
DATASET_PATH = "/Users/noopy/covid19_unknown_spread/dataset"
datasets = glob.glob(f"{DATASET_PATH}/*.csv")
datasets[:5]

['/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_31_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_30_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_17_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_21_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_7_04_.csv']

In [5]:
# get oldest file in the dataset folder
import os, heapq
def newst_files_in_tree(rootfolder, count=1, extension=".csv"):
    return heapq.nlargest(count,
        (os.path.join(dirname, filename)
        for dirname, dirnames, filenames in os.walk(rootfolder)
        for filename in filenames
        if filename.endswith(extension)),
        key=lambda fn: os.stat(fn).st_mtime)

In [6]:
newst_csv = newst_files_in_tree(DATASET_PATH)[0]

In [7]:
df_temp = pd.read_csv(newst_csv, encoding="utf-8")
df_temp.sample(15)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
2237,2014,15411,8.16.,용산구,-,타시도 확진자 접촉,퇴원
1302,2949,17467,8.23.,동대문구,-,여의도 순복음교회 관련,NaN
357,3894,20035,8.31.,중랑구,-,기타 확진자 접촉,NaN
2345,1906,15237,8.15.,성북구,-,성북구 사랑제일교회 관련,퇴원
897,3354,18655,8.26.,송파구,-,성북구 사랑제일교회 관련,NaN
1473,2778,17126,8.22.,양천구,-,확인 중,NaN
1076,3175,17941,8.25.,서초구,-,성북구 사랑제일교회 관련,NaN
1112,3139,18237,8.25.,강서구,-,기타 확진자 접촉,NaN
3602,649,10828,5.08.,중구,-,이태원 클럽 관련,퇴원
586,3665,19624,8.29.,강서구,-,기타 확진자 접촉,NaN


In [8]:
# overwrite whatever cell value that contains "확인" as "확인 중"
df_temp.loc[df_temp["접촉력"].str.contains("확인"),"접촉력"] = "확인 중"

# check whether those two arrays are combined
df_temp.loc[df_temp["접촉력"].str.contains("확인"),"접촉력"].unique()

array(['확인 중'], dtype=object)

In [9]:
df_temp["접촉력"] = df_temp["접촉력"].str.replace("관련", "")
df_temp["접촉력"] = df_temp["접촉력"].str.strip()

In [10]:
infection_paths = df_temp["접촉력"].unique()
print(len(infection_paths))
infection_paths

223


array(['송파구 소재 병원', '노원구 빛가온교회', '강동구 소재 병원', '성북구 사랑제일교회', '8.15도심집회',
       '확인 중', '도봉구 운동시설', '타시도 확진자 접촉', '기타 확진자 접촉', '다래경매', '노원구 기도모임',
       '동작구 카드 발급업체', '광진구 소재병원', '성북구 체대입시', '성북구 요양시설', '중구 소재 은행',
       '중랑구 소재 체육시설', '관악구 판매업소', '동작구 소재 서울신학교', '용인시 우리제일교회',
       '노원구 손해보험', '구로구 보성운수', '강서구 서울대효요양병원', '서초구 장애인교육시설', '서대문구 지인모임',
       '강북구 일가족', '중구 하나은행본점', '강서구 보안회사', '롯데리아 종사자 모임', '해외 접촉 추정',
       '8.15도심집회(순복음 강북교회)', '영등포구 권능교회', '구로구 아파트', '강남구 소재 아파트',
       '제주 게스트하우스', '성북구 벧엘장로교회', '중앙보훈병원', '관악구 김혜근의원', '동대문구 sk탁구클럽',
       '동작구 스터디카페', '극단 산', 'KT가좌지사', '군인권센터', '동작구 요양시설', '여의도 순복음교회',
       '중구 보험회사(현대해상)', '강서구 병원', '종로구 혜화경찰서', '양천구 되새김교회', '현대커머셜',
       '8.15도심집회(녹색병원)', '영등포 IFC몰 오케스트로', '고양시 반석교회(케네디상가)', '롯데 자산개발',
       '골드트레인', '한양대병원', '은평구 헤어콕', '순복음 강북교회', '서대문구 지인 모임', '강동구 어린이집',
       '광화문집회', '은평구 성경공부모임', '관악구 요양병원', '고대 안암병원', '마포구 푸본생명콜센터',
       '성동구 가족', '양천구 되새김 교회', '중구 통일상가', '고양시 반석교회  (케네디상가)',
       '강남구 판매업소(

In [11]:
df_date = df_temp.sort_values(["연번"], ascending=False)
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,9.04.,타시,-,송파구 소재 병원,NaN
1,4250,20979,9.04.,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,9.04.,성동구,-,강동구 소재 병원,NaN
3,4248,20962,9.04.,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,9.03.,성북구,-,8.15도심집회,NaN


In [13]:
df_date["확진일"] = df_date["확진일"].str.replace(".", "-")
df_date["확진일"] = df_date["확진일"].str[:-1]
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,9-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,9-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,9-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,9-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,9-03,성북구,-,8.15도심집회,NaN


In [14]:
df_date["확진일"] = "2020-0" + df_date["확진일"]
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [15]:
df_date.sample(5)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
38,4213,20963,2020-09-04,관악구,-,동작구 카드 발급업체,NaN
3243,1008,11842,2020-06-08,구로구,-,리치웨이,퇴원
866,3385,18575,2020-08-26,성북구,-,확인 중,NaN
3031,1220,12425,2020-06-21,도봉구,-,요양시설,퇴원
2865,1386,13268,2020-07-08,서대문구,미국,해외 접촉 추정,퇴원


In [16]:
df = df_date.copy()
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [17]:
df.loc[(df["확진일"].str.len() < 10), '확진일'] = df.loc[(df["확진일"].str.len() < 10), '확진일'].str.replace("-0", "-")
df.loc[(df["확진일"].str.len() < 10), '확진일'] = df.loc[(df["확진일"].str.len() < 10), '확진일'].str.replace("-", "-0")
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [18]:
df.sample(15)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
1586,2665,16455,2020-08-21,성북구,-,극단 산,NaN
184,4067,20521,2020-09-02,은평구,-,기타 확진자 접촉,NaN
2335,1916,15123,2020-08-15,성북구,-,성북구 사랑제일교회,NaN
1992,2259,16010,2020-08-18,서초구,-,골드트레인,퇴원
782,3469,18973,2020-08-27,강남구,-,기타 확진자 접촉,NaN
2559,1692,14634,2020-08-10,영등포구,-,확인 중,퇴원
2917,1334,12919,2020-07-01,동대문구,-,확인 중,퇴원
1897,2354,15998,2020-08-18,기타,-,성북구 사랑제일교회,NaN
2697,1554,14081,2020-07-24,송파구,-,송파구 지인모임,퇴원
590,3661,19416,2020-08-29,강북구,-,기타 확진자 접촉,NaN


In [19]:
df.to_csv("./dataset_predict/df_wrangle.csv", index=False)

In [20]:
temp = df["확진일"].value_counts().rename_axis('date').reset_index(name='new_confirmed')
temp.head()

,date,new_confirmed
0,2020-08-26,153
1,2020-08-18,151
2,2020-08-15,146
3,2020-08-27,146
4,2020-08-22,140


In [21]:
df_predict = temp.sort_values(by="date")
df_predict.head()

,date,new_confirmed
188,2020-01-24,1
170,2020-01-30,3
164,2020-01-31,3
183,2020-02-02,1
175,2020-02-05,2


In [22]:
df_predict["date"] = pd.to_datetime(df_predict["date"])

In [23]:
df_predict.index = pd.DatetimeIndex(df_predict["date"])
df_predict.head()

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1
2020-01-30,2020-01-30,3
2020-01-31,2020-01-31,3
2020-02-02,2020-02-02,1
2020-02-05,2020-02-05,2


In [24]:
df_predict = df_predict.resample('D').max()
df_predict.head()

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1.0
2020-01-25,NaT,NaN
2020-01-26,NaT,NaN
2020-01-27,NaT,NaN
2020-01-28,NaT,NaN


In [25]:
df_predict["date"] = df_predict.index
df_predict.head()

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1.0
2020-01-25,2020-01-25,NaN
2020-01-26,2020-01-26,NaN
2020-01-27,2020-01-27,NaN
2020-01-28,2020-01-28,NaN


In [26]:
df_predict = df_predict.fillna(0)
df_predict

,date,new_confirmed
date,,
2020-01-24,2020-01-24,1.0
2020-01-25,2020-01-25,0.0
2020-01-26,2020-01-26,0.0
2020-01-27,2020-01-27,0.0
2020-01-28,2020-01-28,0.0
...,...,...
2020-08-31,2020-08-31,94.0
2020-09-01,2020-09-01,101.0
2020-09-02,2020-09-02,81.0


In [27]:
df_predict.to_csv("dataset_predict/df_predict.csv", index=False)

In [28]:
df_predict =  pd.read_csv("dataset_predict/df_predict.csv", encoding="utf-8")

In [29]:
df_untracked = df[df["접촉력"]=="확인 중"]
df_untracked

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
5,4246,미부여,2020-09-03,관악구,-,확인 중,NaN
6,4245,미부여,2020-09-03,마포구,-,확인 중,NaN
7,4244,미부여,2020-08-30,기타,-,확인 중,NaN
8,4243,미부여,2020-09-02,마포구,-,확인 중,NaN
9,4242,미부여,2020-09-03,성동구,-,확인 중,NaN
...,...,...,...,...,...,...,...
4195,56,1768,2020-02-25,고양시,-,확인 중,퇴원
4209,42,1370,2020-02-25,노원구,-,확인 중,퇴원
4212,39,924,2020-02-25,평택,-,확인 중,퇴원
4213,38,907,2020-02-25,관악구,-,확인 중,퇴원


In [30]:
df_untracked_temp = df_untracked.groupby(by=["확진일"]).size()
df_untracked_temp = df_untracked_temp.rename_axis('date').reset_index(name='new_untracked')
df_untracked_temp

,date,new_untracked
0,2020-02-25,6
1,2020-02-26,1
2,2020-02-27,2
3,2020-02-28,2
4,2020-03-02,2
...,...,...
119,2020-08-31,16
120,2020-09-01,18
121,2020-09-02,17
122,2020-09-03,11


In [31]:
df_untracked_no = df_untracked_temp

In [32]:
df_merged = pd.merge(df_predict, df_untracked_no, how="left", on="date")
df_merged = df_merged.fillna(0)
df_merged.head(15)

,date,new_confirmed,new_untracked
0,2020-01-24,1.0,0.0
1,2020-01-25,0.0,0.0
2,2020-01-26,0.0,0.0
3,2020-01-27,0.0,0.0
4,2020-01-28,0.0,0.0
5,2020-01-29,0.0,0.0
6,2020-01-30,3.0,0.0
7,2020-01-31,3.0,0.0
8,2020-02-01,0.0,0.0
9,2020-02-02,1.0,0.0


In [33]:
df_merged.tail(15)

,date,new_confirmed,new_untracked
210,2020-08-21,128.0,32.0
211,2020-08-22,140.0,44.0
212,2020-08-23,97.0,25.0
213,2020-08-24,134.0,45.0
214,2020-08-25,113.0,34.0
215,2020-08-26,153.0,45.0
216,2020-08-27,146.0,34.0
217,2020-08-28,127.0,35.0
218,2020-08-29,114.0,20.0
219,2020-08-30,95.0,20.0


In [49]:
# list_infection_paths_no = []
for index, row in df_merged.iterrows():
    filter_end_date = row["date"]
    # print(filter_end_date)
    df_filtered = df.loc[df['확진일'] <= filter_end_date]
    
    # 확진자 접촉력이 밝혀진 경우
    infection_paths_known = df_filtered["접촉력"].unique()
    
    # "기타 확진자 접촉" 항목 485건 / R0
    contact_no = len(df_filtered[df_filtered["접촉력"].str.contains("기타 확진자 접촉")])/1.5
    
    infection_paths_no = len(infection_paths_known) + contact_no
    list_infection_paths_no.append(int(infection_paths_no))
print(list_infection_paths_no[:10])
print(list_infection_paths_no[-10:])

[1, 1, 1, 1, 1, 1, 2, 4, 4, 4]
[364, 394, 426, 450, 462, 479, 495, 518, 530, 546]


In [54]:
df_merged["no_paths"] = list_infection_paths_no
df_merged["new_confirmed"] = df_merged["new_confirmed"].apply(int)
df_merged["new_untracked"] = df_merged["new_untracked"].apply(int)
df_merged.head()

,date,new_confirmed,new_untracked,no_paths
0,2020-01-24,1,0,1
1,2020-01-25,0,0,1
2,2020-01-26,0,0,1
3,2020-01-27,0,0,1
4,2020-01-28,0,0,1


In [55]:
df_merged.tail()

,date,new_confirmed,new_untracked,no_paths
220,2020-08-31,94,16,479
221,2020-09-01,101,18,495
222,2020-09-02,81,17,518
223,2020-09-03,62,11,530
224,2020-09-04,45,6,546


In [57]:
df_merged.to_csv("dataset_predict/df_predict.csv", index=False)

### check holiday

In [100]:
import json

list_calendar=[]

calendar_file = "./korean-calendar/korean-calendar.json"
# read file
with open(calendar_file, 'r') as myfile:
    data=myfile.readlines()

data = [x.strip() for x in data] 

for i in data:
    calendar_data = i[43:]
    calendar_item = "{"+calendar_data
    calendar_item_json = json.loads(calendar_item)
    list_calendar.append(calendar_item_json)
list_calendar[:10]

[{'sc': '2000-01-01', 'lc': '1999-11-25', 'w': 7, 'h': True, 'ht': '신정'},
 {'sc': '2000-01-02', 'lc': '1999-11-26', 'w': 1, 'h': False, 'ht': ''},
 {'sc': '2000-01-03', 'lc': '1999-11-27', 'w': 2, 'h': False, 'ht': ''},
 {'sc': '2000-01-04', 'lc': '1999-11-28', 'w': 3, 'h': False, 'ht': ''},
 {'sc': '2000-01-05', 'lc': '1999-11-29', 'w': 4, 'h': False, 'ht': ''},
 {'sc': '2000-01-06', 'lc': '1999-11-30', 'w': 5, 'h': False, 'ht': ''},
 {'sc': '2000-01-07', 'lc': '1999-12-01', 'w': 6, 'h': False, 'ht': ''},
 {'sc': '2000-01-08', 'lc': '1999-12-02', 'w': 7, 'h': False, 'ht': ''},
 {'sc': '2000-01-09', 'lc': '1999-12-03', 'w': 1, 'h': False, 'ht': ''},
 {'sc': '2000-01-10', 'lc': '1999-12-04', 'w': 2, 'h': False, 'ht': ''}]

In [129]:
df_calendar = pd.DataFrame(list_calendar)
df_calendar = df_calendar[["sc", "w", "h"]]
df_calendar.head()

,sc,w,h
0,2000-01-01,7,True
1,2000-01-02,1,False
2,2000-01-03,2,False
3,2000-01-04,3,False
4,2000-01-05,4,False


In [139]:
def label_weekend(row):
    # sunday
    if row['w'] == 1:
        return True
    # friday
    if row['w'] == 6:
        return True
    # saturday
    if row['w'] == 7:
        return True
    else:
        return False

In [140]:
df_calendar["is_weekend"] = df_calendar.apply(lambda row: label_weekend(row), axis=1)
df_calendar.head(10)

,date,w,is_holiday,is_weekend
0,2000-01-01,7,True,True
1,2000-01-02,1,False,True
2,2000-01-03,2,False,False
3,2000-01-04,3,False,False
4,2000-01-05,4,False,False
5,2000-01-06,5,False,False
6,2000-01-07,6,False,True
7,2000-01-08,7,False,True
8,2000-01-09,1,False,True
9,2000-01-10,2,False,False


In [141]:
# rename dataframe name
df_calendar.rename(columns = {'sc':'date', 'h':'is_holiday'}, inplace = True)

In [142]:
df_calendar.to_csv("dataset_predict/df_calendar.csv", index=False)

In [143]:
df_calendar_subset = df_calendar[["date", "is_holiday", "is_weekend"]]

In [151]:
df_train = pd.merge(df_merged, df_calendar_subset, how="left", on="date")
df_train.head()

,date,new_confirmed,new_untracked,no_paths,is_holiday,is_weekend
0,2020-01-24,1,0,1,True,True
1,2020-01-25,0,0,1,True,True
2,2020-01-26,0,0,1,True,True
3,2020-01-27,0,0,1,False,False
4,2020-01-28,0,0,1,False,False


In [150]:
df_train[-25:-15]

,date,new_confirmed,new_untracked,no_paths,is_holiday,is_weekend
200,2020-08-11,15,4,159,False,False
201,2020-08-12,26,5,163,False,False
202,2020-08-13,33,1,169,False,False
203,2020-08-14,73,4,175,False,True
204,2020-08-15,146,6,181,True,True
205,2020-08-16,90,4,187,False,True
206,2020-08-17,132,7,191,False,False
207,2020-08-18,151,18,197,False,False
208,2020-08-19,135,19,216,False,False
209,2020-08-20,126,28,229,False,False


In [152]:
df_train.to_csv("./dataset_predict/df_train.csv", index=False)